In [ ]:
pip install python-pptx

# Task 1: Loading and Processing Data from a Powerpoint Presentation

In [5]:
# Step 1: Import the necessary libraries
from langchain_community.document_loaders import UnstructuredPowerPointLoader
import nltk
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown

# Download NLTK resources for text processing
print("Downloading NLTK resources...")
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
print("✅ NLTK resources downloaded successfully")

# Step 2: Load the PowerPoint presentation using UnstructuredPowerPointLoader
print("\n" + "="*60)
print("LOADING POWERPOINT PRESENTATION")
print("="*60)

# Load the PowerPoint presentation using UnstructuredPowerPointLoader
loader = UnstructuredPowerPointLoader('Bitte pitch deck.pptx')
docs = loader.load()

# Confirm successful loading by printing the number of sections loaded
print(f"✅ Presentation loaded with {len(docs)} sections.")

# Step 3: Display the first section of the loaded PowerPoint to verify data
print("\n" + "="*60)
print("VERIFYING LOADED CONTENT")
print("="*60)

# Display the first section of the loaded PowerPoint to verify data
print("📄 First section of the PowerPoint:")
print(docs[0])  # Print the first section of the document

# Optional: Display additional information about all sections
print(f"\n📊 Total sections loaded: {len(docs)}")
for i, doc in enumerate(docs):
    print(f"Section {i+1}: {len(doc.page_content)} characters")
    if doc.page_content:
        print(f"Preview: {doc.page_content[:100]}...")
    print("-" * 40)

✅ NLTK resources downloaded successfully

LOADING POWERPOINT PRESENTATION


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\yaseen_banu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\yaseen_banu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


✅ Presentation loaded with 1 sections.

VERIFYING LOADED CONTENT
📄 First section of the PowerPoint:
page_content='BITTE

AI-Powered Customer Service Revolution

Series A Funding Pitch

2024



Company Overview

Our Mission

To revolutionize the way people interact with technology through innovative solutions.

Our Product

An AI-powered platform that helps businesses automate customer service while maintaining human touch.

Unique Selling Point

99.9% accuracy in understanding customer intent

Seamless integration with existing systems

Reduces customer service costs by 60%



Meet Our Founders

Sarah Johnson - CEO

Former Google Product Manager with 10 years experience

Led product development for Google Assistant

Michael Chen - CTO

Ex-Microsoft Engineer, AI specialist

Published 15+ papers on machine learning

Emma Rodriguez - CMO

Marketing expert from Apple

Grew Apple's B2B segment by 300%



Market Analysis

Target Market

Mid to large enterprises with customer service teams

E

# Task 2: Process and Embed PowerPoint Chunks

In [6]:

print("\n" + "="*60)
print("SPLITTING DOCUMENT INTO CHUNKS")
print("="*60)

# Step 1: Initialize the text splitter with a chunk size of 500 and 50 characters overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Split the document into chunks
chunks = text_splitter.split_documents(docs)

# Verify the number of chunks created
print(f"✅ Number of chunks: {len(chunks)}")

# Optional: Display chunk information
if chunks:
    print(f"📊 Chunk size range: {min(len(chunk.page_content) for chunk in chunks)} - {max(len(chunk.page_content) for chunk in chunks)} characters")
    print(f"📄 First chunk preview:")
    print(f"'{chunks[0].page_content[:150]}...'")

print("\n" + "="*60)
print("LOADING EMBEDDING MODEL")
print("="*60)

# Step 2: Load the embedding model from Hugging Face
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Print confirmation of the embedding model loading
print("✅ Hugging Face embedding model loaded successfully.")

print("\n" + "="*60)
print("CREATING FAISS VECTOR DATABASE")
print("="*60)

# Step 3: Embed the document chunks as vectors and store them in a FAISS vector database
db_faiss = FAISS.from_documents(chunks, embedding_model)

# Confirm that the data has been stored successfully
print("✅ Document chunks embedded and stored in FAISS vector database.")

# Optional: Display vector database information
print(f"📊 Number of vectors in FAISS database: {db_faiss.index.ntotal}")
print(f"📐 Vector dimension: {embedding_model.client[1].get_sentence_embedding_dimension()}")

print("\n" + "="*60)
print("POWERPOINT PROCESSING COMPLETED")
print("="*60)


SPLITTING DOCUMENT INTO CHUNKS
✅ Number of chunks: 4
📊 Chunk size range: 463 - 493 characters
📄 First chunk preview:
'BITTE

AI-Powered Customer Service Revolution

Series A Funding Pitch

2024



Company Overview

Our Mission

To revolutionize the way people interact...'

LOADING EMBEDDING MODEL


C:\Users\yaseen_banu\AppData\Local\Temp\ipykernel_40048\781152356.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\yaseen_banu\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Hugging Face embedding model loaded successfully.

CREATING FAISS VECTOR DATABASE
✅ Document chunks embedded and stored in FAISS vector database.
📊 Number of vectors in FAISS database: 4
📐 Vector dimension: 384

POWERPOINT PROCESSING COMPLETED


# Task 3: Create Document Retrieval Function

In [7]:

print("\n" + "="*60)
print("CREATING DOCUMENT RETRIEVAL FUNCTION")
print("="*60)

# Step 1: Define a function to retrieve relevant documents based on a query
def retrieve_docs(query, k):
    # Perform similarity search on the FAISS database using the query
    docs_faiss = db_faiss.similarity_search(query, k=k)
    
    # Return the most relevant document chunks
    return docs_faiss

print("✅ Document retrieval function created successfully.")

print("\n" + "="*60)
print("TESTING RETRIEVAL FUNCTION")
print("="*60)

# Step 2: Test the function with a sample query and retrieve top 5 results
context = retrieve_docs('Who are the founders?', 5)

# Display the first retrieved chunk to verify correct retrieval
print(f"📄 Query: 'Who are the founders?'")
print(f"📊 Number of chunks retrieved: {len(context)}")
print(f"\n🔍 First retrieved chunk:")
print(context[0])

print("\n" + "="*60)
print("DISPLAYING ALL RETRIEVED CHUNKS")
print("="*60)

# Optional: Display all retrieved chunks with their content
for i, doc in enumerate(context, 1):
    print(f"\n📄 Chunk {i}:")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 40)


CREATING DOCUMENT RETRIEVAL FUNCTION
✅ Document retrieval function created successfully.

TESTING RETRIEVAL FUNCTION
📄 Query: 'Who are the founders?'
📊 Number of chunks retrieved: 4

🔍 First retrieved chunk:
page_content='Meet Our Founders

Sarah Johnson - CEO

Former Google Product Manager with 10 years experience

Led product development for Google Assistant

Michael Chen - CTO

Ex-Microsoft Engineer, AI specialist

Published 15+ papers on machine learning

Emma Rodriguez - CMO

Marketing expert from Apple

Grew Apple's B2B segment by 300%



Market Analysis

Target Market

Mid to large enterprises with customer service teams

E-commerce companies

SaaS businesses

Healthcare organizations

Market Size' metadata={'source': 'Bitte pitch deck.pptx'}

DISPLAYING ALL RETRIEVED CHUNKS

📄 Chunk 1:
Content: Meet Our Founders

Sarah Johnson - CEO

Former Google Product Manager with 10 years experience

Led product development for Google Assistant

Michael Chen - CTO

Ex-Microsoft Engineer, 

# Task 4: Generate Marketing Campaign Responses

In [12]:

print("\n" + "="*60)
print("SETTING UP MARKETING CAMPAIGN GENERATOR")
print("="*60)

from langchain_openai import AzureChatOpenAI
import os

from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from dotenv import load_dotenv

# Load env vars
load_dotenv()

llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0
)

print("✅ ChatOpenAI client initialized successfully.")

# Step 2: Define the user query and system prompt
# Define the user query
query = "a fun interactive way to showcase the brand's unique selling point"

# Retrieve relevant context based on the user query
context = retrieve_docs(query, 10)

# Define the system prompt for the assistant
system_message = f"""
        You are a marketing expert. Based on the theme and context provided, create a tailored marketing campaign for a brand.
        The campaign should include: target audience, campaign strategy, and goals.
        Answer the {query} with the {context} provided.
    """

print("✅ Query and system message defined.")
print(f"✅ Retrieved {len(context)} relevant document chunks for context.")

# Step 3: Structure the messages for the assistant
messages = [("system", system_message), ("human", query)]

print("✅ Messages structured for the assistant.")

print("\n" + "="*60)
print("GENERATING MARKETING CAMPAIGN")
print("="*60)

# Step 4: Generate and display the marketing campaign
# Generate and display the response from the assistant
response = llm.invoke(messages)  # Call the API with the messages

# Display the response in markdown format
display(Markdown(response.content))  # Display the response in markdown format

print("\n" + "="*60)
print("MARKETING CAMPAIGN GENERATED SUCCESSFULLY")
print("="*60)


SETTING UP MARKETING CAMPAIGN GENERATOR
✅ ChatOpenAI client initialized successfully.
✅ Query and system message defined.
✅ Retrieved 4 relevant document chunks for context.
✅ Messages structured for the assistant.

GENERATING MARKETING CAMPAIGN


**Campaign Title: "The AI Customer Service Challenge"**

**Target Audience:**
- Mid to large enterprises with customer service teams
- E-commerce companies
- SaaS businesses
- Healthcare organizations

**Campaign Strategy:**
1. **Interactive Online Challenge:**
   - Create an engaging online game where participants can experience the AI-powered customer service platform in action. The game will simulate real customer service scenarios where players must choose the best responses to customer inquiries.
   - Players will be scored based on their ability to understand customer intent accurately, mirroring the 99.9% accuracy of Bitte's platform.
   - Incorporate fun elements like time limits, bonus points for quick responses, and "power-ups" that represent the platform's features (e.g., multi-language support, sentiment analysis).

2. **Social Media Integration:**
   - Encourage participants to share their scores on social media using a dedicated hashtag (e.g., #AIChallengeWithBitte).
   - Offer weekly prizes for the highest scores, such as discounts on the service, branded merchandise, or a free consultation session.

3. **Webinars and Live Demos:**
   - Host live webinars where potential clients can see the platform in action, showcasing its seamless integration and cost-saving benefits.
   - Include a Q&A session where attendees can ask questions about the technology and its applications in their industries.

4. **Customer Testimonials:**
   - Feature video testimonials from existing clients who have successfully implemented Bitte's platform, highlighting the accuracy and cost savings they experienced.

**Goals:**
- Increase brand awareness and engagement among target audiences.
- Generate leads by encouraging participants to sign up for a demo after completing the challenge.
- Showcase the unique selling point of 99.9% accuracy in understanding customer intent in a fun and memorable way.
- Position Bitte as a thought leader in AI-powered customer service solutions.

**Fun Interactive Element:**
- At the end of the challenge, participants receive a personalized report detailing their performance, including tips on improving customer service interactions. This report can also include a comparison of their scores with the average score of businesses using Bitte's platform, emphasizing the effectiveness of the AI solution.
- Additionally, participants can unlock a "virtual badge" for completing the challenge, which they can display on their LinkedIn profiles, further promoting the brand and its innovative approach to customer service. 

This campaign not only highlights Bitte's unique selling point but also engages potential clients in a fun and interactive way, making the brand memorable and relatable.


MARKETING CAMPAIGN GENERATED SUCCESSFULLY
